In [2]:
import tensorflow as tf

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model


In [15]:
def mobilenet_block(x, filters, strides):
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = ReLU()(x)
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = ReLU()(x)
    return x

In [16]:

#stem of the model
input = Input(shape = (224,224,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)
x = mobilenet_block(x, filters = 64, strides = 1)
x = mobilenet_block(x, filters = 128, strides = 2)
x = mobilenet_block(x, filters = 128, strides = 1)
x = mobilenet_block(x, filters = 256, strides = 2)
x = mobilenet_block(x, filters = 256, strides = 1)
x = mobilenet_block(x, filters = 512, strides = 2)
for _ in range (5):
     x = mobilenet_block(x, filters = 512, strides = 1)
x = mobilenet_block(x, filters = 1024, strides = 2)
x = mobilenet_block(x, filters = 1024, strides = 1)
x = AvgPool2D(pool_size = 7, strides = 1, data_format='channels_first')(x)
output = Dense(units = 1000, activation = 'softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_43 (ReLU)              (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d_19 (Depthwi (None, 112, 112, 32)      320       
_________________________________________________________________
re_lu_44 (ReLU)              (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 112, 112, 64)      2112  

In [17]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [18]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

model.compile(optimizer = "adam", loss = loss_fn, metrics = ["accuracy"])

In [19]:
model.fit(train_images, train_labels, epochs= 10)

Epoch 1/10


ValueError: in user code:

    /Users/jiwon_hae/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/jiwon_hae/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/jiwon_hae/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/jiwon_hae/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/jiwon_hae/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/jiwon_hae/opt/ana